<a href="https://colab.research.google.com/github/TIO4100-IFO26/master-thesis/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**FUNKSJONER SOM BRUKES AV ALLE MODELLER**

In [8]:
def process():
  dataset = pd.read_excel("/content/master-thesis/PredictorData2020.xlsx", sheet_name ="FirstSet")
  dataset['ERP_(t-1)'] = dataset['ERP'].shift(1)
  dataset['ERP_(t+1)'] = dataset['ERP'].shift(-1)
  #from 900:1790 are the dates 194601 to 202002
  dataset = dataset.iloc[900:1790]
  return dataset

def update_predictors(dataset):
  data = dataset.reset_index(drop=True).drop(['yyyymm'], axis=1)
  for column in data:
    if data[column].count()<60:
      data = data.drop([column], axis=1)
    else:
      data[column]=data[column].interpolate(method='linear', limit_direction='backward')
  return data